In [ ]:
# import os
# import json
# from collections import defaultdict
# from tqdm import tqdm

# def parse_cord_v2_json(json_path):
#     with open(json_path, 'r', encoding='utf-8') as f:
#         data = json.load(f)

#     grouped = defaultdict(lambda: {"count": "", "name": "", "price": ""})

#     total_price = ""
    
#     for line in data["valid_line"]:
#         category = line["category"]
#         group_id = line["group_id"]
#         words = " ".join(w["text"] for w in line["words"])

#         if category == "menu.cnt":
#             grouped[group_id]["count"] = words
#         elif category in ["menu.nm", "menu.sub_nm"]:
#             grouped[group_id]["name"] += words + " "
#         elif category == "menu.price":
#             grouped[group_id]["price"] = words
#         elif category == "total.total_price":
#             total_price = words

#     items = []
#     for g in sorted(grouped.keys()):
#         entry = grouped[g]
#         name = entry["name"].strip()
#         if name and entry["price"]:
#             items.append({
#                 "name": name,
#                 "count": entry["count"],
#                 "price": entry["price"]
#             })

#     result = {
#         "items": items,
#         "total": total_price
#     }

#     return result

# def prepare_donut_cord_dataset(json_dir, image_dir, output_path):
#     dataset = []

#     json_files = [f for f in os.listdir(json_dir) if f.endswith(".json")]

#     for json_file in tqdm(json_files, desc="Processing JSON files"):
#         json_path = os.path.join(json_dir, json_file)
#         image_name = os.path.splitext(json_file)[0] + ".png"  # nếu là .jpg sửa ở đây
#         image_path = os.path.join(image_dir, image_name)

#         if not os.path.exists(image_path):
#             print(f"Image not found for {json_file}, skipping...")
#             continue

#         text_output = parse_cord_v2_json(json_path)
        
#         dataset.append({
#             "id": json_file.replace(".json", ""),
#             "image_path": image_path,
#             "text_input": "<s_cord-v2>",  # prefix task
#             "text_output": json.dumps(text_output, ensure_ascii=False)
#         })

#     with open(output_path, "w", encoding="utf-8") as f:
#         json.dump(dataset, f, ensure_ascii=False, indent=2)

#     print(f"✅ Done! Saved {len(dataset)} samples to {output_path}")

# # Cách chạy
# prepare_donut_cord_dataset(
#     json_dir="data/json",
#     image_dir="data/image",
#     output_path="data/train.json"
# )


In [ ]:
# import os

# image_dir = "data/images"
# annotation_dir = "data/annotations"

# # Kiểm tra thư mục tồn tại
# print("Thư mục images tồn tại:", os.path.exists(image_dir))
# print("Thư mục annotation tồn tại:", os.path.exists(annotation_dir))

# # Liệt kê các file trong thư mục
# print("File trong images:", os.listdir(image_dir))
# print("File trong annotation:", os.listdir(annotation_dir))

In [ ]:
# import os
# import json
# import shutil
# from sklearn.model_selection import train_test_split

# # Định nghĩa thư mục gốc và tỷ lệ chia dữ liệu
# dataset_name = "cord_dataset"
# image_dir = "data/images"  # Thay bằng đường dẫn thực tế nếu cần
# annotation_dir = "data/annotations"  # Thay bằng đường dẫn thực tế nếu cần
# train_ratio, val_ratio, test_ratio = 0.8, 0.1, 0.1

# # Kiểm tra thư mục
# if not os.path.exists(image_dir):
#     raise FileNotFoundError(f"Thư mục {image_dir} không tồn tại")
# if not os.path.exists(annotation_dir):
#     raise FileNotFoundError(f"Thư mục {annotation_dir} không tồn tại")

# # Tạo thư mục cho dataset
# os.makedirs(dataset_name, exist_ok=True)
# for split in ["train", "validation", "test"]:
#     os.makedirs(os.path.join(dataset_name, split), exist_ok=True)

# # Hàm xử lý file JSON để tạo ground truth
# def process_cord_json(json_path):
#     with open(json_path, 'r') as f:
#         data = json.load(f)

#     menu_items = []
#     subtotal = None
#     total_info = {}

#     grouped_lines = {}
#     for line in data["valid_line"]:
#         group_id = line["group_id"]
#         if group_id not in grouped_lines:
#             grouped_lines[group_id] = []
#         grouped_lines[group_id].append(line)

#     for group_id, lines in grouped_lines.items():
#         menu_item = {}
#         for line in lines:
#             category = line["category"]
#             text = " ".join(word["text"] for word in line["words"])

#             if category.startswith("menu"):
#                 if category == "menu.nm":
#                     menu_item["name"] = text
#                 elif category == "menu.cnt":
#                     menu_item["count"] = text
#                 elif category == "menu.price":
#                     menu_item["price"] = text
#             elif category == "sub_total.subtotal_price":
#                 subtotal = text
#             elif category.startswith("total"):
#                 if category == "total.total_price":
#                     total_info["total_price"] = text
#                 elif category == "total.cashprice":
#                     total_info["cash"] = text
#                 elif category == "total.changeprice":
#                     total_info["change"] = text
#                 elif category == "total.menutype_cnt":
#                     total_info["menu_types"] = text
#                 elif category == "total.menuqty_cnt":
#                     total_info["menu_quantity"] = text

#         if menu_item:
#             menu_item.setdefault("count", None)
#             menu_item.setdefault("price", None)
#             menu_items.append(menu_item)

#     ground_truth = {
#         "menu": menu_items,
#         "subtotal": subtotal,
#         "total": total_info
#     }
#     return ground_truth

# # Thu thập tất cả các file JSON và ảnh
# data = []
# for json_file in os.listdir(annotation_dir):
#     if json_file.endswith(".json"):
#         image_file_base = json_file.replace(".json", "")
#         image_path = None
#         for ext in [".jpg", ".png"]:  # Hỗ trợ nhiều định dạng ảnh
#             temp_path = os.path.join(image_dir, image_file_base + ext)
#             if os.path.exists(temp_path):
#                 image_path = temp_path
#                 image_file = image_file_base + ext
#                 break

#         if image_path is None:
#             print(f"Không tìm thấy ảnh cho {json_file}")
#             continue

#         json_path = os.path.join(annotation_dir, json_file)
#         ground_truth = process_cord_json(json_path)
#         data.append({
#             "image_file": image_file,
#             "image_path": image_path,
#             "ground_truth": ground_truth
#         })

# # Kiểm tra xem có dữ liệu không
# if not data:
#     raise ValueError("Không tìm thấy dữ liệu nào để xử lý. Vui lòng kiểm tra thư mục images và annotation.")

# # Chia dữ liệu thành train, validation, test
# train_data, temp_data = train_test_split(data, train_size=train_ratio, random_state=42)
# val_data, test_data = train_test_split(temp_data, test_size=test_ratio/(test_ratio + val_ratio), random_state=42)

# # Hàm lưu dữ liệu vào thư mục và tạo metadata.jsonl
# def save_split(split_data, split_name):
#     split_dir = os.path.join(dataset_name, split_name)
#     metadata_path = os.path.join(split_dir, "metadata.jsonl")

#     with open(metadata_path, "w") as f:
#         for item in split_data:
#             dest_image_path = os.path.join(split_dir, item["image_file"])
#             shutil.copy(item["image_path"], dest_image_path)

#             metadata = {
#                 "file_name": item["image_file"],
#                 "ground_truth": json.dumps({"gt_parse": item["ground_truth"]})
#             }
#             f.write(json.dumps(metadata) + "\n")

# # Lưu dữ liệu vào các thư mục
# save_split(train_data, "train")
# save_split(val_data, "validation")
# save_split(test_data, "test")

# print("Dataset đã được tạo xong!")

In [ ]:
!python test.py --dataset_name_or_path dataset --pretrained_model_name_or_path result_new/train_cord/test_experiment --save_path ./result/output.json


In [ ]:

# parser = argparse.ArgumentParser()
# parser.add_argument("--task", type=str, default="cord-v2")
# parser.add_argument("--pretrained_path", type=str, default="naver-clova-ix/donut-base-finetuned-cord-v2")
# args, left_argv = parser.parse_known_args()


In [ ]:
import argparse
import gradio as gr
import torch
from PIL import Image
from donut import DonutModel

def demo_process(input_img):
    global pretrained_model, task_prompt, task_name
    input_img = Image.fromarray(input_img)
    output = pretrained_model.inference(image=input_img, prompt=task_prompt)["predictions"][0]
    return output

def demo_process_vqa(input_img, question):
    global pretrained_model, task_prompt, task_name
    input_img = Image.fromarray(input_img)
    user_prompt = task_prompt.replace("{user_input}", question)
    output = pretrained_model.inference(input_img, prompt=user_prompt)["predictions"][0]
    return output


In [ ]:
task_name = 'cord'
task_prompt = f"<s_{task_name}>"

pretrained_model = DonutModel.from_pretrained(r"result_new/train_cord/test_experiment")
# pretrained_model = DonutModel.from_pretrained("naver-clova-ix/donut-base-finetuned-cord-v2")

if torch.cuda.is_available():
    pretrained_model.half()
    device = torch.device("cuda")
    pretrained_model.to(device)
else:
    pretrained_model.encoder.to(torch.bfloat16)

# pretrained_model.eval()
demo = gr.Interface(
    fn=demo_process_vqa if task_name == "docvqa" else demo_process,
    inputs=["image", "text"] if task_name == "docvqa" else "image",
    outputs="json",
    title=f"Donut 🍩 demonstration for `{task_name}` task",
)
demo.launch(debug=True)


In [ ]:
import cv2


In [ ]:
pretrained_model = DonutModel.from_pretrained("donut-base-finetuned-docvqa")

def demo_process(input_img):
    global pretrained_model, task_prompt, task_name
    input_img = Image.fromarray(input_img)
    output = pretrained_model.inference(image=input_img, prompt=task_prompt)["predictions"][0]
    return output

image = cv2.imread(r'D:\Private\Project\kaggle\LabAI\Donut\image.png')
demo_process(image)

In [ ]:
import cv2
import os
from PIL import Image
import matplotlib.pyplot as plt
import glob
from donut import DonutModel

# Define the demo_process function (as provided)

pretrained_model = DonutModel.from_pretrained("naver-clova-ix/donut-base-finetuned-cord-v2")

def demo_process(input_img):
    global pretrained_model, task_prompt, task_name
    input_img = Image.fromarray(input_img)
    output = pretrained_model.inference(image=input_img, prompt=task_prompt)["predictions"][0]
    return output

# Directory containing images and output
image_dir = r'D:\Private\Project\kaggle\LabAI\Donut\donut\dataset\train'
output_dir = r'D:\Private\Project\kaggle\LabAI\Donut\donut\dataset\output'

# Create output directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

# Get all image files in the directory (e.g., .jpg files)
image_paths = glob.glob(os.path.join(image_dir, '*.jpg'))

# Process each image
for image_path in image_paths:
    # Load the image
    image = cv2.imread(image_path)
    
    # Check if the image was loaded successfully
    if image is None:
        print(f"Could not load image at {image_path}")
        continue
    
    # Convert BGR to RGB
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Process the image to get the output
    try:
        output = demo_process(image_rgb)
        # Shorten the output (e.g., first 100 characters)
        short_output = str(output)[:100] + ('...' if len(str(output)) > 100 else '')
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        continue
    
    # Create a figure with a larger image and smaller text area
    plt.figure(figsize=(12, 6))
    
    # Display the input image (larger)
    plt.subplot(1, 2, 1)
    plt.imshow(image_rgb)
    plt.title('Input Image')
    plt.axis('off')
    
    # Display the shortened output text (smaller area)
    plt.subplot(1, 2, 2)
    plt.text(0.1, 0.5, short_output, fontsize=10, wrap=True, verticalalignment='center')
    plt.title('Output')
    plt.axis('off')
    
    # Adjust layout to make image dominant
    plt.tight_layout()
    
    # Save the combined visualization
    output_filename = os.path.join(output_dir, f'result_{os.path.basename(image_path)}')
    plt.savefig(output_filename, bbox_inches='tight', dpi=150)
    print(f"Saved result for {image_path} to {output_filename}")
    
    # Close the plot to free memory
    plt.close()

print("Processing complete!")

In [ ]:
!pip install timm einops


In [ ]:
import os
import numpy as np
import torch
import torchvision.transforms as T
# from decord import VideoReader, cpu
from PIL import Image
from torchvision.transforms.functional import InterpolationMode
from transformers import AutoModel, AutoTokenizer

IMAGENET_MEAN = (0.485, 0.456, 0.406)
IMAGENET_STD = (0.229, 0.224, 0.225)

def build_transform(input_size):
    MEAN, STD = IMAGENET_MEAN, IMAGENET_STD
    transform = T.Compose([
        T.Lambda(lambda img: img.convert('RGB') if img.mode != 'RGB' else img),
        T.Resize((input_size, input_size), interpolation=InterpolationMode.BICUBIC),
        T.ToTensor(),
        T.Normalize(mean=MEAN, std=STD)
    ])
    return transform

def find_closest_aspect_ratio(aspect_ratio, target_ratios, width, height, image_size):
    best_ratio_diff = float('inf')
    best_ratio = (1, 1)
    area = width * height
    for ratio in target_ratios:
        target_aspect_ratio = ratio[0] / ratio[1]
        ratio_diff = abs(aspect_ratio - target_aspect_ratio)
        if ratio_diff < best_ratio_diff:
            best_ratio_diff = ratio_diff
            best_ratio = ratio
        elif ratio_diff == best_ratio_diff:
            if area > 0.5 * image_size * image_size * ratio[0] * ratio[1]:
                best_ratio = ratio
    return best_ratio

def dynamic_preprocess(image, min_num=1, max_num=12, image_size=448, use_thumbnail=False):
    orig_width, orig_height = image.size
    aspect_ratio = orig_width / orig_height

    # calculate the existing image aspect ratio
    target_ratios = set(
        (i, j) for n in range(min_num, max_num + 1) for i in range(1, n + 1) for j in range(1, n + 1) if
        i * j <= max_num and i * j >= min_num)
    target_ratios = sorted(target_ratios, key=lambda x: x[0] * x[1])

    # find the closest aspect ratio to the target
    target_aspect_ratio = find_closest_aspect_ratio(
        aspect_ratio, target_ratios, orig_width, orig_height, image_size)

    # calculate the target width and height
    target_width = image_size * target_aspect_ratio[0]
    target_height = image_size * target_aspect_ratio[1]
    blocks = target_aspect_ratio[0] * target_aspect_ratio[1]

    # resize the image
    resized_img = image.resize((target_width, target_height))
    processed_images = []
    for i in range(blocks):
        box = (
            (i % (target_width // image_size)) * image_size,
            (i // (target_width // image_size)) * image_size,
            ((i % (target_width // image_size)) + 1) * image_size,
            ((i // (target_width // image_size)) + 1) * image_size
        )
        # split the image
        split_img = resized_img.crop(box)
        processed_images.append(split_img)
    assert len(processed_images) == blocks
    if use_thumbnail and len(processed_images) != 1:
        thumbnail_img = image.resize((image_size, image_size))
        processed_images.append(thumbnail_img)
    return processed_images

def load_image(image_file, input_size=448, max_num=12):
    image = Image.open(image_file).convert('RGB')
    transform = build_transform(input_size=input_size)
    images = dynamic_preprocess(image, image_size=input_size, use_thumbnail=True, max_num=max_num)
    pixel_values = [transform(image) for image in images]
    pixel_values = torch.stack(pixel_values)
    return pixel_values

In [ ]:
model_name = "5CD-AI/Vintern-1B-v2"
model_name = "5CD-AI/Vintern-1B-v3_5"

In [ ]:
from transformers import AutoConfig
config = AutoConfig.from_pretrained("5CD-AI/Vintern-1B-v3_5", revision="main", trust_remote_code=True)

In [2]:
from datasets import load_dataset
dataset = load_dataset("sroie")  # Kiểm tra xem có sẵn không

DatasetNotFoundError: Dataset 'sroie' doesn't exist on the Hub or cannot be accessed.